In [ ]:
!pip install opencv-python numpy scipy tqdm

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [ ]:
import numpy as np
import cv2
import gradio as gr
from scipy.spatial import distance
from scipy.ndimage import convolve
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import spsolve

In [ ]:
def get_sparse_neighbor(p: int, n: int, m: int):
    """Returns a dictionary of 4-neighbors of `p` in a sparse matrix."""
    i, j = p // m, p % m
    d = {}
    if i - 1 >= 0:
        d[(i - 1) * m + j] = (i - 1, j, 0)
    if i + 1 < n:
        d[(i + 1) * m + j] = (i + 1, j, 0)
    if j - 1 >= 0:
        d[i * m + j - 1] = (i, j - 1, 1)
    if j + 1 < m:
        d[i * m + j + 1] = (i, j + 1, 1)
    return d

In [ ]:
def create_spacial_affinity_kernel(spatial_sigma: float, size: int = 15):
    kernel = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            kernel[i, j] = np.exp(-0.5 * (distance.euclidean((i, j), (size // 2, size // 2)) ** 2) / (spatial_sigma ** 2))
    return kernel

In [ ]:
def compute_smoothness_weights(L, x, kernel, eps=1e-3):
    Lp = cv2.Sobel(L, cv2.CV_64F, int(x == 1), int(x == 0), ksize=1)
    T = convolve(np.ones_like(L), kernel, mode='constant')
    T = T / (np.abs(convolve(Lp, kernel, mode='constant')) + eps)
    return T / (np.abs(Lp) + eps)

In [ ]:
def refine_illumination_map_linear(L, gamma, lambda_, kernel, eps=1e-3):
    wx = compute_smoothness_weights(L, x=1, kernel=kernel, eps=eps)
    wy = compute_smoothness_weights(L, x=0, kernel=kernel, eps=eps)

    n, m = L.shape
    L_1d = L.copy().flatten()

    row, column, data = [], [], []
    for p in range(n * m):
        diag = 0
        for q, (k, l, x) in get_sparse_neighbor(p, n, m).items():
            weight = wx[k, l] if x else wy[k, l]
            row.append(p)
            column.append(q)
            data.append(-weight)
            diag += weight
        row.append(p)
        column.append(p)
        data.append(diag)
    F = csr_matrix((data, (row, column)), shape=(n * m, n * m))

    Id = diags([np.ones(n * m)], [0])
    A = Id + lambda_ * F
    L_refined = spsolve(csr_matrix(A), L_1d).reshape((n, m))

    return np.clip(L_refined, eps, 1) ** gamma

def correct_underexposure(im, gamma, lambda_, kernel, eps=1e-3):
    L = np.max(im, axis=-1)
    L_refined = refine_illumination_map_linear(L, gamma, lambda_, kernel, eps)
    L_refined_3d = np.repeat(L_refined[..., None], 3, axis=-1)
    return im / L_refined_3d

In [ ]:
def enhance_image_exposure(im, gamma, lambda_, sigma=3, eps=1e-3):
    kernel = create_spacial_affinity_kernel(sigma)
    im_normalized = im.astype(float) / 255.
    enhanced_image = correct_underexposure(im_normalized, gamma, lambda_, kernel, eps)
    return np.clip(enhanced_image * 255, 0, 255).astype(np.uint8)

def process_image(file, gamma):
    # Ensure the file is properly read
    if not file:
        return None

    image = cv2.imread(file)  # Read image
    if image is None:
        return None  # If image fails to load, return None

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    enhanced_image = enhance_image_exposure(image, gamma, lambda_=0.15)

    return enhanced_image  # Returning a NumPy array (Gradio supports this)

demo = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type='filepath', label='Upload Image'),
        gr.Slider(0.1, 2.0, value=0.6, step=0.1, label='Gamma')
    ],
    outputs=gr.Image(type="numpy", label='Enhanced Image'),
    title="🌙 Low-Light Image Enhancement ☀️",
    description="Upload a low-light image for enhancement"
)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5900d6a6084ac49bf4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
